# Step 1: Load the Data (Extract)

In [137]:
# Imports
import requests
import os
import pandas as pd
from dotenv import load_dotenv

# Make sure .env is loaded and load the polygon API key
load_dotenv("../../.env")
api_key = os.environ['poly_api']

# Construct the API URLs
wgmi_url = f"https://api.polygon.io/v2/aggs/ticker/WGMI/range/1/day/2023-01-01/2024-12-31?adjusted=false&sort=asc&apiKey={api_key}"
nvda_url = f"https://api.polygon.io/v2/aggs/ticker/NVDA/range/1/day/2023-01-01/2024-12-31?adjusted=true&sort=asc&apiKey={api_key}"
forex_url = f"https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/1/day/2023-01-01/2024-12-31?adjusted=true&sort=asc&apiKey={api_key}"
index_url = f"https://api.polygon.io/v2/aggs/ticker/I:NDX/range/1/day/2023-01-01/2024-12-31?sort=asc&apiKey={api_key}"

# Make requests to API
etf_data = requests.get(wgmi_url).json()
nvda_data = requests.get(nvda_url).json()
forex_data = requests.get(forex_url).json()
index_data = requests.get(index_url).json()

# Convert responses into dataframes
etf_df = pd.json_normalize(etf_data['results'])
nvda_df = pd.json_normalize(nvda_data['results'])
forex_df = pd.json_normalize(forex_data['results'])
index_df = pd.json_normalize(index_data['results'])

# Step 2: Clean the Data (Transform)
Our data cleaning process will be as follows:
1. Look for nulls, unecessary data, etc
2. Give columns more descriptive names
3. Convert values to a more readable format (i.e. unix to datetime)
4. Add data into the dataframe
5. Stack the dataframes into one big dataframe

1. Look for nulls, unecessary data, etc

    First, we will check each dataframe using info()

In [138]:
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   v       381 non-null    float64
 1   vw      381 non-null    float64
 2   o       381 non-null    float64
 3   c       381 non-null    float64
 4   h       381 non-null    float64
 5   l       381 non-null    float64
 6   t       381 non-null    int64  
 7   n       381 non-null    int64  
dtypes: float64(6), int64(2)
memory usage: 23.9 KB


In [139]:
nvda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   v       381 non-null    float64
 1   vw      381 non-null    float64
 2   o       381 non-null    float64
 3   c       381 non-null    float64
 4   h       381 non-null    float64
 5   l       381 non-null    float64
 6   t       381 non-null    int64  
 7   n       381 non-null    int64  
dtypes: float64(6), int64(2)
memory usage: 23.9 KB


In [140]:
forex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   v       501 non-null    int64  
 1   vw      501 non-null    float64
 2   o       501 non-null    float64
 3   c       501 non-null    float64
 4   h       501 non-null    float64
 5   l       501 non-null    float64
 6   t       501 non-null    int64  
 7   n       501 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 31.4 KB


In [141]:
index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   o       348 non-null    float64
 1   c       348 non-null    float64
 2   h       348 non-null    float64
 3   l       348 non-null    float64
 4   t       348 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 13.7 KB


There are no nulls to deal with- but we have 3 columns we don't need. Volume, Volume Weighted Average Price, and Transaction Count. We will remove these.

In [142]:
# Drop unecessary columns
etf_df.drop(columns=['v', 'vw', 'n'], inplace=True)
nvda_df.drop(columns=['v', 'vw', 'n'], inplace=True)
forex_df.drop(columns=['v', 'vw', 'n'], inplace=True)
# Index does not have these columns

2. Give columns more descriptive names

In [143]:
etf_df.rename(columns={
    'o': 'Open Price ($)',
    'c': 'Close Price ($)',
    'h': 'Highest Price ($)',
    'l': 'Lowest Price ($)',
    't': 'Date'
    }, inplace=True)

nvda_df.rename(columns={
    'o': 'Open Price ($)',
    'c': 'Close Price ($)',
    'h': 'Highest Price ($)',
    'l': 'Lowest Price ($)',
    't': 'Date'
    }, inplace=True)

forex_df.rename(columns={
    'o': 'Open Price ($)',
    'c': 'Close Price ($)',
    'h': 'Highest Price ($)',
    'l': 'Lowest Price ($)',
    't': 'Date'
    }, inplace=True)

index_df.rename(columns={
    'o': 'Open Price ($)',
    'c': 'Close Price ($)',
    'h': 'Highest Price ($)',
    'l': 'Lowest Price ($)',
    't': 'Date'
    }, inplace=True)

3. Convert values to a more readable format (i.e. unix to datetime)

    The only value that really needs changed is the datetime.

In [144]:
# Convert Unix to datetime
etf_df['Date'] = pd.to_datetime(etf_df['Date'], unit="ms")
nvda_df['Date'] = pd.to_datetime(nvda_df['Date'], unit="ms")
forex_df['Date'] = pd.to_datetime(forex_df['Date'], unit="ms")
index_df['Date'] = pd.to_datetime(index_df['Date'], unit="ms")

# Format datetime
etf_df['Date'] = etf_df['Date'].dt.strftime('%Y-%m-%d')
nvda_df['Date'] = nvda_df['Date'].dt.strftime('%Y-%m-%d')
forex_df['Date'] = forex_df['Date'].dt.strftime('%Y-%m-%d')
index_df['Date'] = index_df['Date'].dt.strftime('%Y-%m-%d')

4. Add data into the dataframe

    We want the symbol for each stock, as we will combine each data frame into one big dataframe to make it easier to handle in SQL.

In [145]:
# Add in ticker symbols
etf_df.insert(loc=0, column="Symbol", value="WGMI")
nvda_df.insert(loc=0, column="Symbol", value="NVDA")
forex_df.insert(loc=0, column="Symbol", value="C:EURUSD")
index_df.insert(loc=0, column="Symbol", value="NDX")

5. Stack the dataframes into one big dataframe

In [146]:
# Combine all 4 dataframes
df = pd.concat([etf_df, nvda_df, forex_df, index_df], ignore_index=True)

We'll check our data to make sure it looks good-

In [147]:
df.head()

,Symbol,Open Price ($),Close Price ($),Highest Price ($),Lowest Price ($),Date
0,WGMI,4.64,4.5588,4.6400,4.42,2023-01-03
1,WGMI,4.71,5.0974,5.1400,4.71,2023-01-04
2,WGMI,5.16,5.2400,5.2600,5.00,2023-01-05
3,WGMI,5.24,5.2100,5.2700,5.10,2023-01-06
4,WGMI,5.50,5.8500,6.0999,5.50,2023-01-09


In [148]:
df[381:386]

,Symbol,Open Price ($),Close Price ($),Highest Price ($),Lowest Price ($),Date
381,NVDA,14.851,14.315,14.996,14.096,2023-01-03
382,NVDA,14.567,14.749,14.853,14.241,2023-01-04
383,NVDA,14.491,14.265,14.564,14.148,2023-01-05
384,NVDA,14.474,14.859,15.010,14.034,2023-01-06
385,NVDA,15.284,15.628,16.056,15.141,2023-01-09


In [149]:
df[787:792]

,Symbol,Open Price ($),Close Price ($),Highest Price ($),Lowest Price ($),Date
787,C:EURUSD,1.09206,1.08920,1.09296,1.08500,2023-01-26
788,C:EURUSD,1.08931,1.08700,1.09001,1.08370,2023-01-27
789,C:EURUSD,1.08720,1.08710,1.08720,1.08710,2023-01-28
790,C:EURUSD,1.08620,1.08715,1.08725,1.08616,2023-01-29
791,C:EURUSD,1.08716,1.08490,1.09140,1.08380,2023-01-30


In [150]:
df.tail()

,Symbol,Open Price ($),Close Price ($),Highest Price ($),Lowest Price ($),Date
1606,NDX,19995.279063,20186.633459,20186.633459,19995.279063,2024-07-03
1607,NDX,20224.130200,20391.966101,20406.994986,20201.500774,2024-07-05
1608,NDX,20393.887458,20439.541727,20455.380769,20363.370614,2024-07-08
1609,NDX,20504.173451,20453.024024,20543.902898,20395.572412,2024-07-09
1610,NDX,20533.270853,20675.378624,20690.966399,20479.942609,2024-07-10


Everything looks good- it's ready to be aggregated in SQL.

# Step 3: Export and Load into SQL (Load)

In [151]:
# Export to a CSV file
df.to_csv('stockdata.csv', index=False)